In [1]:
# Import necessary libraries
from ultralytics import YOLO
from tensorflow.keras.models import load_model
import numpy as np
import tensorflow as tf
from PIL import Image as PILImage
import io
from IPython.display import Image, display
from PIL import Image as PILImage
import numpy as np
import tensorflow as tf
import io
import os
import pandas as pd
import torch
import json
from torch import float32, tensor
import torchmetrics
import cv2


2023-12-21 17:17:11.939082: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-21 17:17:11.939162: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-21 17:17:11.981367: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# import os
# import shutil

# # Source directory containing images
# src_dir = './TRAIN'

# # Destination directory to copy images
# dst_dir = './TEST'

# # Check if destination directory exists, create it if it doesn't
# if not os.path.exists(dst_dir):
#     os.makedirs(dst_dir)

# # Loop through all files in source directory
# for file_name in os.listdir(src_dir):
#     # Check if file ends with .jpg
#     if len(os.listdir(dst_dir)) == 50:
#         break
#     if file_name.endswith('.jpg'):
#         # Construct full file paths
#         src_path = os.path.join(src_dir, file_name)
#         dst_path = os.path.join(dst_dir, file_name)
#         # Copy file to destination directory
#         shutil.copy(src_path, dst_path)




In [27]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Load the YOLO model
yolo_model = YOLO('./YOLOv8_training/multiclass_best.pt').to(device)
# Load the ResNet50 model
model2 = load_model('./model_resnet50.h5')

# Class mapping
class_mapping = {'Apple Scab Leaf': 0, 'Apple leaf': 1, 'Apple rust leaf': 2, 'Bell_pepper leaf': 3, 'Bell_pepper leaf spot': 4, 'Blueberry leaf': 5, 'Cherry leaf': 6, 'Corn Gray leaf spot': 7, 'Corn leaf blight': 8, 'Corn rust leaf': 9, 'Peach leaf': 10, 'Potato leaf': 11, 'Potato leaf early blight': 12, 'Potato leaf late blight': 13, 'Raspberry leaf': 14, 'Soyabean leaf': 15, 'Squash Powdery mildew leaf': 16, 'Strawberry leaf': 17, 'Tomato Early blight leaf': 18, 'Tomato Septoria leaf spot': 19, 'Tomato leaf': 20, 'Tomato leaf bacterial spot': 21, 'Tomato leaf late blight': 22, 'Tomato leaf mosaic virus': 23, 'Tomato leaf yellow virus': 24, 'Tomato mold leaf': 25, 'Tomato two spotted spider mites leaf': 26, 'grape leaf': 27, 'grape leaf black rot': 28}

# Directory containing images to predict
directory = './datasets(1)/test/images'
# directory = './TEST'

In [28]:
# Define a list of dictionaries
data = []
# Create empty lists to store results
resized_cropped_images = []
predicted_classes = []
preds=[]

In [29]:
import matplotlib.pyplot as plt
%matplotlib inline

In [30]:
df = pd.read_csv('./YOLOv8_training/new_labels.csv')
# df = pd.read_csv('set.csv')

# Define the target size for ResNet50
target_size = (256, 256)
# Process YOLO results
for filename in os.listdir(directory):
    filtered_df = df[df['filename'] == filename]
    results = yolo_model.predict(os.path.join(directory,filename))
    for x in results:
        for r in x:
            # Get the image and bounding box
            im_array = r.plot()  # plot a BGR numpy array of predictions
            im = PILImage.fromarray(im_array[..., ::-1])  # Convert BGR to RGB PIL image
            bbox = r.boxes.data[0, :4].cpu().numpy()  # Get the bounding box

            # Crop the detected object from the image
            cropped_im = im.crop((bbox[0], bbox[1], bbox[2], bbox[3]))

            # Resize the cropped image to the target size
            resized_cropped_im = cropped_im.resize(target_size)
            # plt.imshow(resized_cropped_im)

            # Save the resized and cropped image to a BytesIO object
            img_io = io.BytesIO()
            resized_cropped_im.save(img_io, 'PNG')

            # Append the resized and cropped image to the list
            resized_cropped_images.append(resized_cropped_im)
            predicted_class_index = r.boxes.cls
            predicted_class_label = list(class_mapping.keys())[int(predicted_class_index.cpu().item())]
            predicted_classes.append(predicted_class_label)
            # Convert bounding box to tensor and move to GPU
            bbox_tensor = torch.tensor([[bbox[0], bbox[1], bbox[2], bbox[3]]]).to(device)
            
            # Convert labels to tensor and move to GPU
            label_tensor = torch.tensor([int(predicted_class_index)]).to(device)

            # Convert scores to tensor and move to GPU
            score_tensor = r.boxes.conf
            minind=0
            q=0
            err=1e9
            for(index, row) in filtered_df.iterrows():
                # Access fields using the column names
                temerr=(bbox[0]-row['xmin'])**2+(bbox[1]-row['ymin'])**2+(bbox[2]-row['xmax'])**2+(bbox[3]-row['ymax'])**2
                if(temerr<err):
                    err=temerr
                    minind=q
                q+=1
            print(len(filtered_df), " ", minind)
            if(len(filtered_df)>0):
                data.append({'boxes': torch.tensor([[(filtered_df.iloc[minind]['xmin'].copy()), (filtered_df.iloc[minind]['ymin'].copy()), (filtered_df.iloc[minind]['xmax'].copy()), (filtered_df.iloc[minind]['ymax'].copy())]]).to(device), 'labels': torch.tensor([class_mapping[filtered_df.iloc[minind]['class']]]).to(device)})
                # filtered_df.drop(filtered_df.index[minind], inplace=True)
                preds.append(dict(boxes=bbox_tensor, labels=label_tensor, scores=score_tensor))
    filtered_df=[]
    
    


image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1309.jpg: 640x416 1 Corn leaf blight, 7.5ms
Speed: 1.0ms preprocess, 7.5ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 416)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img2458.jpg: 480x640 7 Tomato leaf mosaic viruss, 9.7ms
Speed: 0.6ms preprocess, 9.7ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)
10   7
10   8
10   2
10   7
10   6
10   9
10   4

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1086.jpg: 416x640 1 grape leaf black rot, 8.1ms
Speed: 1.1ms preprocess, 8.1ms inference, 0.5ms postprocess per image at shape (1, 3, 416, 640)
2   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img952.jpg: 448x640 3 Cherry leafs, 8.2ms
Speed: 1.0ms preprocess, 8.2ms inference, 0.6ms postprocess per image at shape (1, 3, 448, 

Speed: 1.5ms preprocess, 14.6ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)
4   3
4   0
4   1
4   2
4   3
4   3
4   3
4   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img2143.jpg: 480x640 1 grape leaf black rot, 14.2ms
Speed: 1.3ms preprocess, 14.2ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img2266.jpg: 512x640 1 Potato leaf late blight, 13.5ms
Speed: 1.1ms preprocess, 13.5ms inference, 1.2ms postprocess per image at shape (1, 3, 512, 640)
4   1

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img2523.jpg: 640x416 1 Potato leaf late blight, 12.0ms
Speed: 1.2ms preprocess, 12.0ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 416)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img316.jpg: 288x640 

1   0
1   0
1   0
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1966.jpg: 448x640 1 Corn Gray leaf spot, 8.3ms
Speed: 0.9ms preprocess, 8.3ms inference, 0.6ms postprocess per image at shape (1, 3, 448, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1360.jpg: 480x640 1 Potato leaf early blight, 1 Soyabean leaf, 1 Tomato leaf, 10.1ms
Speed: 1.3ms preprocess, 10.1ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)
1   0
1   0
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img229.jpg: 640x640 1 Potato leaf early blight, 1 Tomato leaf late blight, 12.0ms
Speed: 1.5ms preprocess, 12.0ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)
1   0
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1097.jpg: 512x640 6 Strawberry leafs, 9.2ms
Speed: 1.6ms prepro

Speed: 1.2ms preprocess, 9.6ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)
11   1
11   2
11   0
11   5
11   8
11   3
11   4

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img2476.jpg: 544x640 5 Tomato leafs, 9.3ms
Speed: 1.2ms preprocess, 9.3ms inference, 0.5ms postprocess per image at shape (1, 3, 544, 640)
5   0
5   4
5   1
5   2
5   3

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1741.jpg: 640x640 2 Bell_pepper leaf spots, 11.8ms
Speed: 1.0ms preprocess, 11.8ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)
2   0
2   1

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1972.jpg: 448x640 6 Strawberry leafs, 8.0ms
Speed: 0.9ms preprocess, 8.0ms inference, 0.5ms postprocess per image at shape (1, 3, 448, 640)
6   4
6   0
6   3
6   5
6   1
6   2

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/data

Speed: 1.0ms preprocess, 9.3ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 544)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1835.jpg: 640x640 2 Corn leaf blights, 11.7ms
Speed: 1.2ms preprocess, 11.7ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)
2   1
2   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1004.jpg: 544x640 1 Apple leaf, 9.3ms
Speed: 1.4ms preprocess, 9.3ms inference, 0.5ms postprocess per image at shape (1, 3, 544, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img849.jpg: 640x480 1 Corn leaf blight, 9.7ms
Speed: 0.8ms preprocess, 9.7ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 480)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img943.jpg: 640x480 1 Bell_pepper leaf spot, 9.3ms
Speed: 1.6ms preprocess, 9.3ms inf

Speed: 1.8ms preprocess, 11.9ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)
6   0
6   4
6   1
6   2
6   3
6   5
6   5
6   4
6   4
6   4
6   4

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1226.jpg: 480x640 1 Apple Scab Leaf, 9.8ms
Speed: 1.0ms preprocess, 9.8ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img2378.jpg: 384x640 7 Corn leaf blights, 7.7ms
Speed: 0.8ms preprocess, 7.7ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
7   1
7   3
7   2
7   0
7   3
7   5
7   2

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1788.jpg: 480x640 14 Tomato leafs, 9.9ms
Speed: 1.6ms preprocess, 9.9ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)
12   11
12   0
12   3
12   1
12   5
12   6
12   4
12   5
12   1
12   0
12   1
12   8
12   

Speed: 1.2ms preprocess, 9.7ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)
2   0
2   0
2   0
2   0
2   1
2   1
2   1
2   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img2194.jpg: 448x640 1 Tomato leaf late blight, 7.9ms
Speed: 0.8ms preprocess, 7.9ms inference, 0.5ms postprocess per image at shape (1, 3, 448, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1457.jpg: 640x512 5 Bell_pepper leafs, 8.6ms
Speed: 1.1ms preprocess, 8.6ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 512)
5   1
5   0
5   3
5   2
5   4

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1752.jpg: 640x480 6 Tomato Septoria leaf spots, 9.9ms
Speed: 1.6ms preprocess, 9.9ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 480)
4   2
4   3
4   0
4   1
4   2
4   2

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detecti

Speed: 0.6ms preprocess, 9.0ms inference, 0.5ms postprocess per image at shape (1, 3, 512, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1614.jpg: 384x640 1 Corn leaf blight, 7.5ms
Speed: 0.8ms preprocess, 7.5ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1001.jpg: 640x448 1 Apple rust leaf, 12.7ms
Speed: 1.8ms preprocess, 12.7ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 448)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1310.jpg: 544x640 2 Tomato leaf late blights, 14.2ms
Speed: 1.5ms preprocess, 14.2ms inference, 1.3ms postprocess per image at shape (1, 3, 544, 640)
2   1
2   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img273.jpg: 640x576 5 Apple Scab Leafs, 14.0ms
Speed: 1.2ms preprocess

Speed: 1.3ms preprocess, 13.5ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 512)
1   0
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1314.jpg: 640x416 3 Peach leafs, 12.4ms
Speed: 1.5ms preprocess, 12.4ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 416)
3   2
3   0
3   1

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img944.jpg: 448x640 1 Potato leaf early blight, 12.8ms
Speed: 1.5ms preprocess, 12.8ms inference, 1.2ms postprocess per image at shape (1, 3, 448, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1551.jpg: 480x640 1 Blueberry leaf, 5 Soyabean leafs, 14.5ms
Speed: 1.5ms preprocess, 14.5ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)
10   5
10   2
10   0
10   8
10   2
10   5

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1282.

In [31]:
# for img_data, predicted_class in zip(resized_cropped_images, predicted_classes):
#     display(img_data, predicted_class, class_mapping[predicted_class])


In [32]:
metrics = torchmetrics.detection.MeanAveragePrecision().to(device)
for i in range(len(preds)):
    print(preds[i])
    print(data[i])
metrics.update(preds, data)
print(metrics.compute())

{'boxes': tensor([[ 175.0732,    0.0000,  573.0431, 1020.3662]], device='cuda:0'), 'labels': tensor([8], device='cuda:0'), 'scores': tensor([0.8786], device='cuda:0')}
{'boxes': tensor([[ 181,    7,  527, 1024]], device='cuda:0'), 'labels': tensor([8], device='cuda:0')}
{'boxes': tensor([[  0.0000, 160.5174, 208.9548, 284.2183]], device='cuda:0'), 'labels': tensor([23], device='cuda:0'), 'scores': tensor([0.6253], device='cuda:0')}
{'boxes': tensor([[ 25, 160, 204, 279]], device='cuda:0'), 'labels': tensor([23], device='cuda:0')}
{'boxes': tensor([[428.7725,  90.2564, 565.3612, 248.9724]], device='cuda:0'), 'labels': tensor([23], device='cuda:0'), 'scores': tensor([0.5981], device='cuda:0')}
{'boxes': tensor([[437,  83, 563, 237]], device='cuda:0'), 'labels': tensor([23], device='cuda:0')}
{'boxes': tensor([[130.5516,   0.0000, 266.3249, 218.1907]], device='cuda:0'), 'labels': tensor([23], device='cuda:0'), 'scores': tensor([0.5295], device='cuda:0')}
{'boxes': tensor([[152,   7, 238, 

{'boxes': tensor([[236,  24, 436, 204]], device='cuda:0'), 'labels': tensor([15], device='cuda:0')}
{'boxes': tensor([[456.3442,  11.5664, 574.8041, 158.6414]], device='cuda:0'), 'labels': tensor([15], device='cuda:0'), 'scores': tensor([0.9092], device='cuda:0')}
{'boxes': tensor([[236,  24, 436, 204]], device='cuda:0'), 'labels': tensor([15], device='cuda:0')}
{'boxes': tensor([[22.6741,  0.0000, 80.9287, 49.5170]], device='cuda:0'), 'labels': tensor([15], device='cuda:0'), 'scores': tensor([0.9065], device='cuda:0')}
{'boxes': tensor([[ 96, 175, 278, 371]], device='cuda:0'), 'labels': tensor([15], device='cuda:0')}
{'boxes': tensor([[305.2070, 227.6119, 502.5948, 410.7325]], device='cuda:0'), 'labels': tensor([15], device='cuda:0'), 'scores': tensor([0.9053], device='cuda:0')}
{'boxes': tensor([[308, 228, 506, 403]], device='cuda:0'), 'labels': tensor([15], device='cuda:0')}
{'boxes': tensor([[3.8990e-02, 4.9470e+01, 5.6410e+01, 1.5179e+02]], device='cuda:0'), 'labels': tensor([15],

{'boxes': tensor([[224.7006, 148.9396, 414.7607, 273.6067]], device='cuda:0'), 'labels': tensor([17], device='cuda:0'), 'scores': tensor([0.9369], device='cuda:0')}
{'boxes': tensor([[216, 155, 413, 274]], device='cuda:0'), 'labels': tensor([17], device='cuda:0')}
{'boxes': tensor([[455.7899, 132.6514, 666.7567, 254.3179]], device='cuda:0'), 'labels': tensor([17], device='cuda:0'), 'scores': tensor([0.9323], device='cuda:0')}
{'boxes': tensor([[463, 133, 656, 275]], device='cuda:0'), 'labels': tensor([17], device='cuda:0')}
{'boxes': tensor([[199.1648, 224.7876, 346.8195, 384.0874]], device='cuda:0'), 'labels': tensor([17], device='cuda:0'), 'scores': tensor([0.9238], device='cuda:0')}
{'boxes': tensor([[198, 214, 346, 387]], device='cuda:0'), 'labels': tensor([17], device='cuda:0')}
{'boxes': tensor([[ 67.8528, 183.2625, 220.1908, 352.2088]], device='cuda:0'), 'labels': tensor([17], device='cuda:0'), 'scores': tensor([0.9236], device='cuda:0')}
{'boxes': tensor([[ 67, 186, 218, 355]],

{'boxes': tensor([[   3.1115,  931.0401,  421.2219, 1242.3871]], device='cuda:0'), 'labels': tensor([6], device='cuda:0'), 'scores': tensor([0.6226], device='cuda:0')}
{'boxes': tensor([[ 993,   47, 1395,  439]], device='cuda:0'), 'labels': tensor([17], device='cuda:0')}
{'boxes': tensor([[388.0476, 659.5070, 626.8627, 991.9620]], device='cuda:0'), 'labels': tensor([5], device='cuda:0'), 'scores': tensor([0.3766], device='cuda:0')}
{'boxes': tensor([[ 993,   47, 1395,  439]], device='cuda:0'), 'labels': tensor([17], device='cuda:0')}
{'boxes': tensor([[388.5388, 658.7355, 627.3946, 993.7012]], device='cuda:0'), 'labels': tensor([6], device='cuda:0'), 'scores': tensor([0.2507], device='cuda:0')}
{'boxes': tensor([[ 993,   47, 1395,  439]], device='cuda:0'), 'labels': tensor([17], device='cuda:0')}
{'boxes': tensor([[146.2538,  52.2183, 715.8554, 521.3647]], device='cuda:0'), 'labels': tensor([22], device='cuda:0'), 'scores': tensor([0.4107], device='cuda:0')}
{'boxes': tensor([[136,  70

{'boxes': tensor([[410.4537, 288.3071, 472.4379, 369.3336]], device='cuda:0'), 'labels': tensor([5], device='cuda:0'), 'scores': tensor([0.8374], device='cuda:0')}
{'boxes': tensor([[ 67, 125, 130, 186]], device='cuda:0'), 'labels': tensor([13], device='cuda:0')}
{'boxes': tensor([[341.4445, 232.3474, 390.6081, 306.4228]], device='cuda:0'), 'labels': tensor([5], device='cuda:0'), 'scores': tensor([0.8351], device='cuda:0')}
{'boxes': tensor([[ 67, 125, 130, 186]], device='cuda:0'), 'labels': tensor([13], device='cuda:0')}
{'boxes': tensor([[ 95.6801,  19.6415, 165.8898,  98.5489]], device='cuda:0'), 'labels': tensor([12], device='cuda:0'), 'scores': tensor([0.7293], device='cuda:0')}
{'boxes': tensor([[ 93,  20, 167,  96]], device='cuda:0'), 'labels': tensor([13], device='cuda:0')}
{'boxes': tensor([[432.2295, 237.7351, 499.8797, 298.3121]], device='cuda:0'), 'labels': tensor([5], device='cuda:0'), 'scores': tensor([0.6909], device='cuda:0')}
{'boxes': tensor([[ 67, 125, 130, 186]], de

...

{'boxes': tensor([[ 94.9376, 369.4165, 722.7073, 767.8374]], device='cuda:0'), 'labels': tensor([10], device='cuda:0'), 'scores': tensor([0.9442], device='cuda:0')}
{'boxes': tensor([[104, 378, 714, 767]], device='cuda:0'), 'labels': tensor([10], device='cuda:0')}
{'boxes': tensor([[198.8694, 225.8603, 661.4858, 426.5140]], device='cuda:0'), 'labels': tensor([10], device='cuda:0'), 'scores': tensor([0.9239], device='cuda:0')}
{'boxes': tensor([[186, 232, 653, 418]], device='cuda:0'), 'labels': tensor([10], device='cuda:0')}
{'boxes': tensor([[212.4744, 395.0909, 539.1677, 518.2321]], device='cuda:0'), 'labels': tensor([10], device='cuda:0'), 'scores': tensor([0.9004], device='cuda:0')}
{'boxes': tensor([[178, 405, 533, 525]], device='cuda:0'), 'labels': tensor([10], device='cuda:0')}
{'boxes': tensor([[590.8857,  65.8849, 748.0644, 350.0035]], device='cuda:0'), 'labels': tensor([10], device='cuda:0'), 'scores': tensor([0.8291], device='cuda:0')}
{'boxes': tensor([[583,  71, 730, 339]],

{'boxes': tensor([[2635.6509, 1736.2828, 3390.8232, 2522.6753]], device='cuda:0'), 'labels': tensor([22], device='cuda:0'), 'scores': tensor([0.4066], device='cuda:0')}
{'boxes': tensor([[ 392,  964, 2968, 2698]], device='cuda:0'), 'labels': tensor([22], device='cuda:0')}
{'boxes': tensor([[3765.4983, 1987.5710, 4416.0000, 2618.1318]], device='cuda:0'), 'labels': tensor([20], device='cuda:0'), 'scores': tensor([0.3977], device='cuda:0')}
{'boxes': tensor([[ 392,  964, 2968, 2698]], device='cuda:0'), 'labels': tensor([22], device='cuda:0')}
{'boxes': tensor([[640.6818, 279.1020, 903.1133, 559.6475]], device='cuda:0'), 'labels': tensor([22], device='cuda:0'), 'scores': tensor([0.8251], device='cuda:0')}
{'boxes': tensor([[652, 296, 903, 539]], device='cuda:0'), 'labels': tensor([22], device='cuda:0')}
{'boxes': tensor([[392.2506, 540.7722, 694.6419, 900.0000]], device='cuda:0'), 'labels': tensor([22], device='cuda:0'), 'scores': tensor([0.8165], device='cuda:0')}
{'boxes': tensor([[398, 

{'boxes': tensor([[121.0190,   0.0000, 912.5636, 538.8267]], device='cuda:0'), 'labels': tensor([8], device='cuda:0'), 'scores': tensor([0.9051], device='cuda:0')}
{'boxes': tensor([[141,   2, 885, 540]], device='cuda:0'), 'labels': tensor([8], device='cuda:0')}
{'boxes': tensor([[3.5824e+00, 0.0000e+00, 2.8480e+03, 4.2720e+03]], device='cuda:0'), 'labels': tensor([2], device='cuda:0'), 'scores': tensor([0.4813], device='cuda:0')}
{'boxes': tensor([[  20,    1, 2848, 4272]], device='cuda:0'), 'labels': tensor([9], device='cuda:0')}
{'boxes': tensor([[ 42.6802,  74.6341, 191.1389, 363.4331]], device='cuda:0'), 'labels': tensor([22], device='cuda:0'), 'scores': tensor([0.8829], device='cuda:0')}
{'boxes': tensor([[ 41,  87, 202, 352]], device='cuda:0'), 'labels': tensor([22], device='cuda:0')}
{'boxes': tensor([[235.4326,  42.1345, 477.9342, 386.1958]], device='cuda:0'), 'labels': tensor([22], device='cuda:0'), 'scores': tensor([0.8456], device='cuda:0')}
{'boxes': tensor([[236,  40, 474

{'boxes': tensor([[ 191.3788,  269.1470, 1520.8972,  947.7737]], device='cuda:0'), 'labels': tensor([20], device='cuda:0'), 'scores': tensor([0.4104], device='cuda:0')}
{'boxes': tensor([[ 176,  304, 1554,  926]], device='cuda:0'), 'labels': tensor([3], device='cuda:0')}
{'boxes': tensor([[ 194.4339,  272.7222, 1534.8077,  989.3665]], device='cuda:0'), 'labels': tensor([25], device='cuda:0'), 'scores': tensor([0.3234], device='cuda:0')}
{'boxes': tensor([[ 176,  304, 1554,  926]], device='cuda:0'), 'labels': tensor([3], device='cuda:0')}
{'boxes': tensor([[158.5719, 144.6244, 499.3852, 963.2345]], device='cuda:0'), 'labels': tensor([6], device='cuda:0'), 'scores': tensor([0.9401], device='cuda:0')}
{'boxes': tensor([[176, 154, 485, 934]], device='cuda:0'), 'labels': tensor([6], device='cuda:0')}
{'boxes': tensor([[446.6079,   3.1799, 736.0000, 981.0000]], device='cuda:0'), 'labels': tensor([6], device='cuda:0'), 'scores': tensor([0.3278], device='cuda:0')}
{'boxes': tensor([[176, 154, 

{'boxes': tensor([[ 396.4909,  616.1636, 1613.6257, 2527.4290]], device='cuda:0'), 'labels': tensor([5], device='cuda:0'), 'scores': tensor([0.7502], device='cuda:0')}
{'boxes': tensor([[ 362,  625, 1673, 2651]], device='cuda:0'), 'labels': tensor([6], device='cuda:0')}
{'boxes': tensor([[1957.5874,  282.0042, 2791.6362, 1611.9413]], device='cuda:0'), 'labels': tensor([5], device='cuda:0'), 'scores': tensor([0.6898], device='cuda:0')}
{'boxes': tensor([[1951,  210, 2803, 1673]], device='cuda:0'), 'labels': tensor([6], device='cuda:0')}
{'boxes': tensor([[  2.0792,   1.8921, 800.0000, 600.0000]], device='cuda:0'), 'labels': tensor([12], device='cuda:0'), 'scores': tensor([0.7437], device='cuda:0')}
{'boxes': tensor([[  4,   2, 799, 583]], device='cuda:0'), 'labels': tensor([12], device='cuda:0')}
{'boxes': tensor([[  1.2746,   1.0099, 796.0394, 600.0000]], device='cuda:0'), 'labels': tensor([25], device='cuda:0'), 'scores': tensor([0.4145], device='cuda:0')}
{'boxes': tensor([[  4,   2,